In [ ]:
%matplotlib notebook

import os
import pickle
import copy

import autograd.numpy as np
import autograd.numpy.random as npr
npr.seed(12345)

import matplotlib.pyplot as plt
from matplotlib import gridspec
from sklearn.decomposition import PCA
from scipy import linalg

import seaborn as sns
color_names = ["windows blue", "red", "amber", "faded green"]
colors = sns.xkcd_palette(color_names)
sns.set_style("white")
sns.set_context("talk")

import ssm

# Interpreting nonlinear dynamical systems using approximate models

In lecture today, we saw how fitting a recurrent switching linear dynamical system (rSLDS) to neural recording data let us interpret that data using methods from linear dynamical systems.

This is very convenient because we know a lot of ways to study linear dynamical systems, whereas nonlinear systems are a huge mess.

## 📖 Contents of this notebook

🏃 = coding exercises  
🐸 = extra things to experiment with if you have time  
🤔 = thought exercises

1. 🌀 Autonomous dynamical systems ($\dot{x} = f(\mathbf{A}x)$)  
    1.1 Our friend the Poincaré diagram  
    1.2 The `simulators.autonomous` class for simulating autonomous systems  
    1.3 Case study: the Lorenz system  
        1.3.1 Simulating dynamics  
        1.3.2 🏃 Visualizing the flow field  
        1.3.2 🏃 Fitting rSLDS
        1.3.3 🏃 Plotting the ELBO
        1.3.4 🏃 Visualizing the fit flow field
        1.3.5 🏃 Choosing the number of states  
2. 🏎️ Input-driven dynamical systems ($\dot{x} = f(\mathbf{A}x + \mathbf{V}u$))  
    2.1 Simulating a leaky integrator RNN
        2.1.1 Getting to know the dynamics matrix
        2.1.2 🏃 Picking the size of the fit dynamical system
        2.1.3 🏃 Fitting rSLDS with inputs
        2.1.4 🏃 Plotting the ELBO and visualizing the fit flow field
        2.1.5 🏃 Getting the inputs wrong
        2.1.6 🏃 Examining parameters of the fit model
    2.2 🐸 further RNN adventures

**Acknowledgement:** parts of this notebook are adapted from Scott Linderman's Recurrent SLDS notebook, which can be found <a href=https://github.com/lindermanlab/ssm/blob/master/notebooks/4-Recurrent-SLDS.ipynb>here</a>!

# SETUP

Go to [the SSM Github repository](https://github.com/lindermanlab/ssm) and follow the **Installation** instructions:
```
git clone https://github.com/lindermanlab/ssm
cd ssm
pip install numpy cython
pip install -e .
```
Then, download [simulators.py](https://drive.google.com/file/d/1o5zAZ5sKi21kvEDbMGqPzs0QgVnpNPzz/view?usp=sharing) which contains a set of helper functions that simulate the dynamics of dynamical systems. Upload it to your google drive and stick it inside `/content/ssm` (which we created above.)

# 1. 🌀 Autonomous dynamical systems ($\dot{x} = f(\mathbf{A}x)$)  
A dynamical system is called autonomous (or homogeneous) if $\dot{x}=0$ when $x=0$, meaning we're studying its dynamics in the absence of any ongoing external input. Broadly speaking, nonlinear autonomous systems can do one of three things:  
<ol>
    <li> all terms either explode to infinity or decay to 0 (though they can do interesting things along the way)  
    <li> produce oscillations (<a href=https://www.youtube.com/watch?v=jRQAndvF4sM>here's</a> a chemical system doing this)
    <li> produce chaos (first demonstrated in neural net models <a href=https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.61.259>here</a>, a more approachable read <a href=https://journals.aps.org/pre/abstract/10.1103/PhysRevE.82.011903>here</a>.)
</ol>
    

## 1.1 Using the Poincaré diagram to predict the behavior of a linear autonomous system
In a **linear** autonomous system of the form $\dot{x} = \mathbf{A}x$, we are restricted to case 1 above: exploding to +/-infinity or decaying to zero (or hovering at a fixed value, but this requires very precise tuning of the values of $\mathbf{A}$.)

Because the behavior of the system is entirely determined by its dynamics matrix $\mathbf{A}$, we can look at features of $\mathbf{A}$ to understand just what the system will do:
<ol>
    <li>The <a href=https://en.wikipedia.org/wiki/Determinant>Determinant</a> of $\mathbf{A}$, which  is the product of its eigenvalues: $\textrm{det}(\mathbf{A}) = \prod_{i=1}^n \lambda_i$
    <li> The <a href=https://en.wikipedia.org/wiki/Trace_(linear_algebra)>Trace</a> of $\mathbf{A}$, which is the sum of its eigenvalues: $\textrm{tr}(\mathbf{A}) = \sum_{i=1}^n \lambda_i$
</ol>

Given these two values, the Poincaré diagram describes the expected behavior of the system:
<div align=center>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/3b/Stability_Diagram.png/825px-Stability_Diagram.png" width=600>
    Fig: Back to haunt you from undergraduate math, the Poincare diagram captures the space of possible behaviors of linear dynamical systems, given the trace and determinant of their dynamics matrix A.
</div>

This also means that if we want to hand-craft a system to behave a certain way, we just have to mess with its eigenvalues.

## 🖥️ 1.2 `simulators.autonomous` : A Python class for simulating dynamical systems
We'll use the provided python module `simulators.py` to simulate the dynamics of some autonomous and driven dynamical systems. This class has a few helpful functions:

#### `autonomous.set_state(args)` sets the state of the system
> Inputs:
> * `args` is a tuple specifying the value for each state variable in your system. Make sure it's the right size.  
>
> Usage: `lorenz.set_state((x,y,z))` to set the state to $x,y,z$ (see next section).  

#### `autonomous.set_timestep(dt)` sets the simulation timestep
> Inputs:
> * `dt` is a simulation timestep in somewhat arbitrary units.
> This defaults to 0.01, you probably shouldn't have to change it.  

#### `autonomous.dynamics(time, (X))` computes the instantaneous dynamics of the system
> Inputs:  
> * `time` allows for the system equations to include an explicit dependence on time (not the case for Lorenz.)  
> * `X` optionally allows you to specify a system state, otherwise the current state is used.  
>
> Returns:
> * `args` the derivative of the system, i.e. $(\dot{x},\dot{y},\dot{z})$ for Lorenz.  

#### `autonomous.simulate(tmax, (persistent))` simulates the system dynamics for `time` seconds
> Inputs:  
> * `persistent` (defaults to `False`) will cause the system state to remain at the value at the end of the simulation, otherwise the system state is unchanged (so two runs of the system will produce identical trajectories.)  
>
> Returns:
> * `args`: the state variables of the system on all simulation timesteps (default timestep is `model.dt = 0.01`)

#### `autonomous.plot_simulation(tmax, (axes, pca))` plots a simulation of the system
> Inputs:
> * `tmax` is how long to simulate.
> * `axes` lets you pass a set of `matplotlib` axes on which to plot the system trajectory. Otherwise makes a new figure.
> * `pca` (defaults to `False`) instructs the model to plot the first 3 PCs of the system when True, otherwise only the first 3 dimensions are plotted.

#### `autonomous.plot_timeseries(tmax, (axes, normalize))` plots each system dimension as a function of time
> Same arguments as `plot_simulation` with the exception of:
> * `normalize` (defaults to `True`) scales each plotted dimension to be bounded on the interval $[0,1]$

#### `autonomous.animate_simulation(tmax, (axes, pca, step))` makes an animated version of `plot_simulation`
> Same arguments as `plot_simulation` with the exception of:
> * `step` (defaults to 50) is the animation time step in units of $dt$. If it's too small you'll run out of memory.

#### `autonomous.observe_simulation(n_obs, (sigma, transform))` makes a noisy `n_obs`-dimensional observation of the system simulation
> We'll use this to experiment with how rSLDS (or other models) performs when given noisy observations of the underlying dynamical system it is fitted to.
> Inputs:
> * `n_obs` is the number of observations to make. Each observation is related to the system variables as $y_n = \sum_{i=1}^D w_i x_i$, where $w_i \sim \mathcal{N}(0,1/\sqrt{D})$ and $D$ is the number of system variables.
> * `sigma` (defaults to 0) is the variance of an added Gaussian noise term. Default is no noise added.
> * `transform` (defaults to 'linear') lets you apply a nonlinear transform to your observations. Options are `linear`, `nonneg`, and `tanh`.

## 1.3 Autonomous system case study: the Lorenz Attractor

The Lorenz attractor (not <a href=https://amenteemaravilhosa.com.br/wp-content/uploads/2018/04/konrad-lorenz-1.jpg>that Lorenz</a>) is a well-known autonomous nonlinear dynamical system that produces chaotic dynamics for certain parameter settings. It is characterized by a system of three equations $\dot{x},\dot{y},\dot{z}$ with three fixed parameters $\sigma, \rho, \beta$:

\begin{align}
\dot{x} & = (y-x)\sigma \\
\dot{y} & = \rho x - y - xz \\
\dot{z} & = -\beta z + xy
\end{align}

By default the model sets $\sigma=10, \beta=2.667,$ and $rho=28$, which produces the iconic chaotic butterfly. Let's take a look using some of the built-in functions of the `simulator` class:

### 1.3.1 Simulating the dynamics of the Lorenz system
For this section I've given you the required code, just to make sure everyone's on the right page:

In [ ]:
os.chdir('/content/ssm')
!ls
import numpy as np

In [ ]:
%matplotlib notebook
import simulators
import numpy as np
import matplotlib.pyplot as plt

lorenz = simulators.lorenz()

# Plot the Lorenz attractor using a Matplotlib 3D projection.
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(121, projection='3d', aspect='auto')

lorenz.plot_simulation(tmax=100, ax=ax, pca=False)

ax2 = fig.add_subplot(122)
lorenz.plot_timeseries(ax=ax2, normalize=True)
plt.show()

And just for fun, we can animate these dynamics as well, to get a visual impression of how they depend on the system state. Note, this animation projects the dynamics onto their first two principal components:

In [ ]:
from matplotlib import rc
rc('animation', html='jshtml')

fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(111, aspect='auto')
plt.xlim(-30, 30)
plt.ylim(-30, 30)

lorenz.simulate(10, persistent=True);  # setting persistent=True "burns-in" to get the system into its steady-state behavior
anim = lorenz.animate_simulation(tmax=20, ax=ax, step=50); # and animation! this takes a bit of time to process
anim

### 1.3.2 🏃 Visualizing the true flow field of the Lorenz system
Before we go fitting any fancy models to these dynamics, we should see what the true underlying flow field of the system looks like.

Complete the code section below to generate an approximate 2D flow field of the system when we project it onto its first two PCs. Some functions that will help you along the way:
* `sklearn.decomposition.PCA` it's, uh, PCA
* `lorenz.step((x,y,z))` will return the derivative of the model at the point $(x,y,z)$. But how do we get $(x,y,z)$ from a point
* `ax.quiver` to plot a flow-field

In [ ]:
from sklearn.decomposition import PCA

# setting up a grid of points at which to compute the flow field:
xlim = [-35, 35]
ylim = [-30, 30]
xx = np.linspace(*xlim, 35)
yy = np.linspace(*ylim, 30)
Xpts, Ypts = np.meshgrid(xx, yy)
xy = np.column_stack((Xpts.ravel(), Ypts.ravel()))

# sample the dynamics of the system to find its PCs


# compute the projection of the flow onto the first 2 PCs


🐸 Of course, we didn't have to project onto the first 2 PCs of our data to make the flow field. What other kinds of 2D flow field can you generate?  
🐸 It turns out that `ax.quiver` does also allow you to plot a 3D flow field. These are usually too messy to get much visual intuition from- can you figure out any tricks to visualize the full 3D flow field of the Lorenz system?


### 🤔 Looking at your flow field- how many linear systems would you use to approximate it? What would their dynamics do?
I bolded this 🤔 so you don't miss it, because up next we're going to find out what rSLDS does.

In [ ]:
ncells = 50
raster = lorenz.observe_simulation(ncells, tmax=100, sigma=0)

plt.figure(figsize=(8,6))
plt.imshow(raster.T,aspect='auto')
plt.title('Some noisy observations of a Lorenz system')
plt.colorbar();

### 1.3.3 Fitting Recurrent SLDS to the Lorenz system
First, a refresher on the rSLDS from the original notebook by Scott Linderman. This section reproduces some text from the [original rSLDS tutorial notebook](https://github.com/lindermanlab/ssm/blob/master/notebooks/4-Recurrent-SLDS.ipynb) for some background:

> A Recurrent Switching Linear Dynamical System (rSLDS) is a generalization of a Switching Linear Dynamical System (SLDS) in which the switches in the discrete state are allowed to depend on the value of the continuous state (hence the name recurrent). The rSLDS was developed by Linderman _et al_ in ["Bayesian Learning and Inference in Recurrent Switching Linear Dynamical Systems"](http://proceedings.mlr.press/v54/linderman17a.html).
>
>### Generative model for rSLDS
The generative model for rSLDS is the same as the SLDS case, except that the discrete state transition probabilities are modulated by the continuous state.
>
>1. **Discrete State Update**. At each time step, sample a new discrete state $z_t \mid z_{t-1}, x_{t-1}$ with probabilities driven by a logistic regression on the continuous state:
$$
p(z_t = i \mid z_{t-1} = j, x_{t-1}) \propto
\exp{\left( \log (P_{j,i}) + W_i^T u_t + R_i ^T x_{t-1} \right)}
$$
where $W_i$ is a vector of weights associated with discrete state $i$, that control dependence on an external, known input $u_t$. $R_i$ is again a vector of weights assocaited with state $i$, which weights the contribution from the prior state.
>
>
> 2. **Continuous State Update**. Update the state using the dynamics matrix corresponding to the new discrete state:
$$
x_t = A_k x_{t-1} + V_k u_{t} + b_k + w_t
$$
$A_k$ is the dynamics matrix corresponding to discrete state $k$. $u_t$ is the input vector (specified by the user, not inferred by SSM) and $V_k$ is the corresponding control matrix. The vector $b$ is an offset vector, which can drive the dynamics in a particular direction.
The terms $w_t$ is a noise terms, which perturbs the dynamics.
Most commonly these are modeled as zero-mean multivariate Gaussians,
but one nice feature of SSM is that it supports many distributions for these noise terms. See the Linear Dynamical Systems notebook for a list of supported dynamics models.
>
>
> 3. **Emission**. We now make an observation of the state, according to the specified observation model. In the general case, the state controls the observation via a Generalized Linear Model:
$$
y_t \sim \mathcal{P}(\eta(C_k x_t + d_k + F_k u_t + v_t))
$$
$\mathcal{P}$ is a probabibility distribution. The inner arguments form an affine measurement of the state, which is then passed through the inverse link function $\eta(\cdot)$.
In this case, $C_k$ is the measurement matrix corresponding to discrete state $k$, $d_k$ is an offset or bias term corresponding to discrete state $k$, $F_k$ is called the feedthrough matrix or passthrough matrix (it passes the input directly to the emission). In the Gaussian case, the emission can simply be written as $y_t = C_k x_t + d_k + F_k u_t + v_t$ where $v_t$ is a Gaussian r.v. See the Linear Dynamical System notebook for a list of the observation models supported by SSM.
  






### Types of transition models
The most general form of the discrete state transition model is the following:
$$
p(z_t = i \mid z_{t-1} = j, x_{t-1}) \propto
\exp{\left( \log (P_{j,i}) + w_i^T u_t + r_i ^T x_{t-1} \right)}
$$

(we write $\propto$ and not $=$ just because we normalize these probabilities to sum to 1.) The term in the exponent has three parts:
1. $\log (P_{j,i})$ is a transition probability term, akin to the transition matrix of a Hidden Markov model.
2. $w_i^T u_t$ is an external input term- our Lorenz system is autonomous, so this will just be zero for us.
3. $ r_i ^T x_{t-1}$ is the "recurrent" part of the model- it allows the current latent state of the system $x$ to influence the probability of changing to a different discrete state.

When fitting any SLDS model, SSM provides several kinds of transition models. These are implemented as classes in `ssm/transitions.py`. The model `transitions` parameter can be set to the following strings:
* `standard` the typical HMM transition matrix (ie only term 1 above).
* `sticky` as above, but with a prior to bias the model towards self-transitions (🤔 why would that be useful?)
* `input_driven` implements the transition matrix as a GLM applied to the external input term (modified version of term 2 above).
* `recurrent` includes terms 1 and 3 above (and term 2 if there is input present.)
* `recurrent_only` replaces term 1 above with a constant "bias" term, so that transition probabilities are only dependent on the state of the system $x$.

One more 🤔 from Scott's tutorial:
>What happens as the magnitude of the entries in $R_i$ become very large (compared to the entries of $R_j$ for the other states? Do the transitions become more or less random?  

### Types of dynamics models
The latent dynamics take the form of an autoregressive process:
$$
x_t = A_k x_{t-1} + V_k u_{t} + b_k + w_t
$$
This again is the most general form of the model, and other, simpler forms are available. SSM provides the following dynamics model options, which are implemented in `ssm/observations.py` and specified by setting the model `dynamics` parameter. These mostly vary in their noise assumptions, and are less critical to play with:
* `gaussian` assumes the latent state dynamics evolve as above, and treats Gaussian observation noise with a full positive definite covariance matrix. This is usually avoided because it's a lot of paramaeters to fit for little gain.
* `diagonal_gaussian` is as above, but observation noise is assumed to be independent between latent variables (ie covariance matrix is constrained to be diagonal.)


### Types of emissions models
The emissions model defines how the latent dynamics are translated to observed (e.g., neural) activity. Most broadly, it takes the form:
$$
y_t \sim \mathcal{P}(\eta(C_k x_t + d_k + F_k u_t + v_t))
$$
SSM provides may forms of the emissions model, which are fond in `ssm/emissions.py` and specified by setting the model `emissions` parameter. These all extend a set of linear emissions base models:
* **Standard**: $E[y | x] = Cx + d + Fu$ is a simple linear mapping from x and u to y, with no constraints on model parameters.
* **Orthogonal**: the emissions matrix $C$ is constrained to be orthogonal.
* **Identity**: the emissions matrix $C$ is the identity matrix, so the observed variables are just noisy versions of the latents.

Each of these three flavors can be applied to various nonlinear emissions model. To do so, change the model specification string `XXX` to `XXX_orthog` or `XXX_id` to use the orthogonality or identity constraints, respectively (eg `gaussian_orthog` and `gaussian_id`.) Here are some emissions models you could try:
* `gaussian` is the standard observation model, assuming Gaussian observation noise.
* `studentst` is the Student's T distribution, which allows heavier tails than the standard Gaussian.
* `poisson` is... yes, Poisson. Good for spiking data, perhaps?
* `bernoulli` for Bernoulli variables, again potentially useful for spiking.

### The actual fitting part
Now, let's create a new rSLDS object and fit it to the data sampled from our Lorenz system. Note that we're only giving the system access to the output of `observe_simulation`, it doesn't get to see the true Lorenz system variables.

We'll be using the `"laplace_em"` fitting method, which requires that we use `"structured_meanfield"` for the `variational_posterior` argument. There's more information on this in section 4.1 of the <a href=https://github.com/lindermanlab/ssm/blob/master/notebooks/4-Recurrent-SLDS.ipynb>rSLDS notebook</a> but the tldr is that of two implemented fitting methods for SLDS, Laplace Variational EM (`"laplace_em"`) consistently converges faster.

The fitting process involves three steps: creating the model, initializing it to sensible values, and then running fitting. We do these with the following functions:

#### `ssm.SLDS(D_obs, K, D_latent, (transitions, dynamics, emissions, single_subspace)` creates the model object
> Inputs:
> * `D_obs` is the number of observed dimensions.
> * `K` is the number of fit discrete states.
> * `D_latent` is the number of fit latent dimensions.
> * `transitions` specifies the transition model (see above). Recommend `recurrent_only`.
> * `dynamics` specifies the dynamics model (see above). Recommend `diagonal_gaussian`.
> * `emissions` specifies-- you guessed it!-- the emissions model (see above). Recommend `gaussian_orthog`.
>
> Outputs:
> * an ssm model object of type SLDS (rSLDS is a subtype of the SLDS class.)

#### `model.initialize(data)` initializes model parameters to sensible values
> This is a surprisingly important step! Discrete models often get stuck in local minima, so when we fit the rSLDS we're actually going to start off by fitting an autoregressive Hidden Markov Model to get our estimates of the `transitions` and `dynamics` matrices. Take a look at `ssm/lds.py` to see what happens here.

#### `model.fit(data, (params))` fits the model, as you might expect
> Inputs:
> * `data` is your observed system state.
> * `method` is the fitting algorithm, keep this to `"laplace_em"`.
> * `variational_posterior` is the method for approximating the posterior of the data, keep this to `"structured_meanfield"`.
> * `num_iters` specifies how long to run fitting.  
>
> Outputs:
> * `elbos` is the Evidence Lower Bound (ELBO) on the log-likelihood of the data at each iteration of fitting, to check convergence.
> * `posterior` is a list of the posterior estimates of latent variables $x(t)$ and discrete states $z(t)$ given the data $y(t)$ used for fitting. Each list entry is one trial- so if you're fitting to a single recording just take the index `[0]`.


### 🏃 Time to fit
Now, let's generate some observation data from our Lorenz system, and fit our model! Here we can check the quality of the observation data quickly:

In [ ]:
ncells = 20  # however many you like
lorenz = simulators.lorenz()  # start with a clean slate
raster = lorenz.observe_simulation(ncells, tmax=100, sigma=3)  # simulate your model, potentially with noisy observations

pca_data   = PCA(n_components=2)
pcs_data   = pca_data.fit_transform(raster)

fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(111)
plt.plot(pcs_data[:,0], pcs_data[:,1], lw=1)
plt.title('first 2 PCs of data')

Now complete the code section below using the three functions above, making sure to save the `elbos` and `posteriors` outputted by the fitting process.

The `model.fit` step takes 4 to 5 seconds per iteration on a laptop (using ncells=20 and tmax=100), and often converges within 50 iterations- so let's start there.

In [ ]:
# Global parameters you'll need to set:
D_obs = _                # number of observed dimensions
D_latent = _             # number of latent dimensions - up to you!
K = _                    # number of latent states to fit - up to you!

# Now fit that model:


### 🏃 1.3.3 Checking Convergence
Okay that was fun- but what just happened? To see if things worked, you should first take a look at the [evidence lower bound (ELBO)](https://en.wikipedia.org/wiki/Evidence_lower_bound) to confirm that the fitting process has converged.

Write some code to do so below. **Note**: the ELBO on the very first fitting step is usually awful, so you should drop it and just plot from the second step onward.

In [ ]:
# plot the ELBO to check for convergence

### 1.3.4 Visualizing the fit model
Assuming the model converged, we should take a look at the fit it produced. You've had to figure out how to plot a fit flow field once already, so for this part you can just use these helper functions.

(One note, the size of the flow-field arrows can be compared within a state, but due to `quiver`'s built-in scaling you can't actually compare them between states.)

In [ ]:
def plot_trajectory(z, x, ax=None, ls="-"):
    zcps = np.concatenate(([0], np.where(np.diff(z))[0] + 1, [z.size]))
    if ax is None:
        fig = plt.figure(figsize=(4, 4))
        if len(x.shape)==3:
            ax = fig.gca(projection='3d')
        else:
            ax = fig.gca()
    for start, stop in zip(zcps[:-1], zcps[1:]):
        if len(x.shape)==3:
            ax.plot(x[start:stop + 1, 0],
                    x[start:stop + 1, 1],
                    x[start:stop + 1, 2],
                    lw=1, ls=ls,
                    color=colors[z[start] % len(colors)],
                    alpha=0.4)
        else:
            ax.plot(x[start:stop + 1, 0],
                    x[start:stop + 1, 1],
                    lw=1, ls=ls,
                    color=colors[z[start] % len(colors)],
                    alpha=0.4)
    return ax


def plot_flattened_dynamics(model, pca, trajectory, xlim=None, ylim=None, nxpts=20, nypts=20, alpha=0.8, ax=None, figsize=(3, 3)):

    PCs   = pca.transform(trajectory)
    if xlim is None:
        xlim = (min(PCs[:,0])-1, max(PCs[:,0])+1)
        ylim = (min(PCs[:,1])-1, max(PCs[:,1])+1)

    x = np.linspace(*xlim, nxpts)
    y = np.linspace(*ylim, nypts)
    X, Y = np.meshgrid(x, y)
    xy = np.column_stack((X.ravel(), Y.ravel()))

    fullD = pca.inverse_transform(xy)

    # Get the probability of each state at each xy location
    # Note that this assumes the `recurrent_only` form of the transition model-
    # if you change that you may have to improvise.
    z = np.argmax(fullD.dot(model.transitions.Rs.T) + model.transitions.r, axis=1)

    if ax is None:
        fig = plt.figure(figsize=[7,7])
        ax = fig.add_subplot(111)
    plt.plot(PCs[:,0], PCs[:,1], lw=0.75, color='k', alpha=0.4)

    # project our D-dimensional dynamics vector onto the first 2 PCs
    for k, (A, b) in enumerate(zip(model.dynamics.As, model.dynamics.bs)):
        dxydt_m = pca.transform((fullD.dot(A.T) + b - fullD))

        zk = z == k
        if zk.sum(0) > 0:
            ax.quiver(xy[zk, 0], xy[zk, 1], dxydt_m[zk, 0], dxydt_m[zk, 1], color=colors[k % len(colors)], alpha=1.0)

    plt.tight_layout()

    return ax

In [ ]:
# TODO: get the values of xhat and zhat
xhat = _
zhat = _

# everything else here you can keep as is:

# get the first 2 PC's of the "neural" data
pca_data   = PCA(n_components=2)
pcs_data   = pca_data.fit_transform(raster)

# get the first 2 PC's of the latent dimensions of the fit model
pca_model  = PCA(n_components=2)
pcs_model  = pca_model.fit_transform(xhat)


# and plot everything!
fig = plt.figure(figsize=(8,8))

# plot the original system
ax = fig.add_subplot(221)
plt.plot(pcs_data[:,0], pcs_data[:,1], lw=1)
plt.title('first 2 PCs of data')

# plot the inferred latent trajectories and states
ax2 = fig.add_subplot(222)
plot_trajectory(zhat, pcs_model, ax=ax2)
plt.title('most-likely state of fit latents')

# add some plots here to compare the PCs of the observed data to those of the fit system:
ax2 = fig.add_subplot(212)
plt.plot(pcs_data[0])
plt.plot(pcs_model[0])


fig2 = plt.figure(figsize=(8,8))
ax = fig2.add_subplot(111)
plot_flattened_dynamics(model, pca_model, xhat, ax=ax)
plt.title('fit flow field');

### 🐸 More questions...
Once you get things working, you can experiment- how does your fit model change if you alter:  
* the number of observed "neurons" `ncells`?
* the amount of observed data `tmax`?
* the observation noise `sigma`?  
* the observation nonlinearity `transform`?  

### 1.3.5 🏃 Choosing the number of discrete states

One fit model is nice, but we had to make a couple guesses to get there- in particular, the number of discrete states $K$.

Rather than guessing a value of $K$, a more principled approach is to fit models with 1, 2, 3... states and compare the ELBO for each state count. Modify the code below to perform this fitting:

In [ ]:
# Global parameters
D_obs = raster.shape[1]  # number of observed dimensions
D_latent = 3             # number of latent dimensions
k_range = range(1,6)     # range of K's to test

# initialize some variables to store info from the fitted models
models     = [0]*len(k_range)  # store the fitted models
elbos      = [0]*len(k_range)  # store the ELBO at each iteration
posteriors = [0]*len(k_range)  # store the posterior objects

# and fit! this takes a few minutes per K, so go stretch your legs and get yourself some coffee


Having fit all those models, the next step is to compare them and settle on a value of $K$! (Note, in practice this can be a bit noisy so you may end up running multiple iterations for each $K$ and taking the best one.)

How do you decide which model is best? Make some plots below and pick your favorite fit.

In [ ]:
# look at ELBOs for each of your fit models!

### Visualizing observed and inferred systems
As above, we'll use the `plot_trajectory` and `plot_flattened_dynamics` helper functions to see the inferred states of our best-fit model.

In [ ]:
use_K = _   # pick your favorite model!

# everything below here you can run as-is

# get the inferred states and latent dimensions from your fit model
model = models[k_range.index(use_K)]
xhat = posteriors[k_range.index(use_K)].mean_continuous_states[0]
zhat = model.most_likely_states(xhat, raster)

# visualizing a high-D system is messy, so let's just look at the first 2 PC's:
# get the first 2 PC's of the "neural" data
pca_data   = PCA(n_components=2)
pcs_data   = pca_data.fit_transform(raster)

# get the first 2 PC's of the latent dimensions of the fit model
pca_model = PCA(n_components=2)
pcs_model   = pca_model.fit_transform(xhat)

# and plot everything!
fig = plt.figure(figsize=(8,4))

# plot the original system
ax = fig.add_subplot(121)
plt.plot(pcs_data[:,0], pcs_model[:,1], lw=1)
plt.title('first 2 PCs of data')

# plot the inferred latent trajectories and states
ax2 = fig.add_subplot(122)
plot_trajectory(zhat, pcs_model, ax=ax2)
plt.title('first 2 PCs of fit latents')

fig2 = plt.figure(figsize=(8,8))
ax = fig2.add_subplot(111)
plot_flattened_dynamics(model, pca_model, xhat, ax=ax)
plt.title('fit flow field')

### 🐸 Yet more questions...
Your bet-fit model and number of states may differ from your neighbors'- these systems are still sensitive to initial conditions. Some other experiments to try
* can you modify this code to use a different `transitions` model? How could you visualize that output?
* does changing the `emissions` model help at all to correct for nonlinear observations of the model state?

In [ ]:
# once you're ready to move on to part 2, you'll probably want to clear your notebook kernel.
# here's a copy of the original import cell at the top of this notebook, since it has been a while.

%matplotlib notebook

import os
import pickle
import copy

import autograd.numpy as np
import autograd.numpy.random as npr
npr.seed(12345)

import matplotlib.pyplot as plt
from matplotlib import gridspec
from sklearn.decomposition import PCA
from scipy import linalg

import seaborn as sns
color_names = ["windows blue", "red", "amber", "faded green"]
colors = sns.xkcd_palette(color_names)
sns.set_style("white")
sns.set_context("talk")

import ssm

# 2. 🏎️ Input-driven dynamical systems ($\dot{x} = f(\mathbf{A}x + \mathbf{V}u$))

The `simulators` library also includes support for input-driven (i.e. non-homogeneous) dynamical systems. You can add inputs to an input-driven system with the following functions:

#### `model.add_pulse(time, (amplitude, weights))`: Creates a brief pulse of width $10dt$ (dt is the simulation timestep).
> Inputs:
> * `time` specifies when the input pulse is delivered, in seconds.
> * `amplitude` (default 1) sets pulse amplitude. This gets scaled by $1/(10dt)$ to account for the simulation timestep.
> * `weights` (default 1 to all units) sets the input weight onto the dimensions of the system.

#### `model.add_step(time, (time_off, amplitude, weights))`: Creates a step input.
> Inputs:
> * `time` specifies when the input step turns on, in seconds.
> * `time_off` (default $\infty$) specifies when the input step turns off, in seconds.
> * `amplitude` (default 1) sets step amplitude. Unlike `add_pulse`, the step amplitude is not scaled by dt.
> * `weights` (default 1 to all units) sets the input weight onto the dimensions of the system.

#### `model.add_input_noise(amplitude, (weights))`: Creates an ongoing white-noise input.
> Note, there is no onset time here, as noise is added to every step of the simulation.
> Inputs:
> * `amplitude` specifies the variance of the noise; this is scaled by $\sqrt{1/dt}$ to make units a bit more similar to those of the input pulses.
> * `weights` (default 1 to all units) sets the weight onto the dimensions of the system.

#### `model.clear_inputs()` removes all added inputs from the model.
> If inputs aren't removed, they'll continue to be applied for all repeat simulations, even if persistent=True in your simulation call.

#### `model.plot_simulation()` modified.
> This function behaves as before, but now includes the input to the model plotted below the state variables.

### The `simulators.RNN` model
To gain some intuition for how state space models work with neural data, let's try running them on some simulated data! The `RNN` model adds just a few additional elements on top of the standard input-driven model:

#### `simulators.RNN((n_dims))` model initialization.
> Optional input `n_dims` specifies the number of latent dimensions of the toy model.
> Note that if you initialize $\mathbf{A}$ to be random, the behavior of the model will be a bit more stable if n_dims is a bit larger (e.g. default is 20), however this doesn't mean that the system's true latent dimensionality is that large.
>
> Alternatively, make `n_dims` smaller and make $\mathbf{A}$ upper-triangular or diagonal for a system that's a bit easier to work with.

#### `model.A` is the dynamics matrix.
> You can reassign the values of $A$  to change the dynamics of the system.

#### `model.dynamics(time, (X))` computes model dynamics.
> Inputs are the same in the autonomous model, but now the dynamics have the form  $ \dot{x} = f( \mathbf{A} x + \textrm{input}) $, where `input` consists of the pulses, steps, and noise terms you add to the model.

#### `model.set_nonlinearity(nonlin)` applies a nonlinearity to the dynamics.
> * `nonlin` is a string or a `lambda`; currently supports these options:
>     * `'linear'`: f(x) = x
>     * `'nonneg'`: f(x) = max(x,0)
>     * `'tanh'`: f(x) = tanh(x)
>     * or you can pass a python `lambda` of your own design.

In [ ]:
%matplotlib notebook
import simulators
import numpy as np
import matplotlib.pyplot as plt

n_dims = 20
integrator = simulators.RNN(n_dims=n_dims)

# let's create a dynamics matrix with one integration dimension
# drawing Gaussian-distributed weights with a variance that scales with 1/sqrt(n_dims)
# ensures that scaling up or down n_dims doesn't dramatically change our system (in the
# limit where n_dims is large.)
integrator.A = np.random.normal(0, 0.5/np.sqrt(n_dims), (n_dims, n_dims))
integrator.A[0,0] = 0.975  # make an integration dimension in our dynamics matrix


integrator.set_nonlinearity('linear')

### 2.1.1 Getting to know the dynamics matrix
Recall that our model's dynamics matrix $\mathbf{A}$ determines how our system behaves (if it's linear). Before we do any simulation, let's take a look at the eigenvalues of the dynamics matrix we constructed above.

In [ ]:
e, v = np.linalg.eig(integrator.A)

fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(111)
plt.plot(e.real, e.imag,'.')
plt.xlabel('real part')
plt.ylabel('imaginary part')
plt.axvline(x=1, color='r', linestyle='--');

🤔 What does the red line represent?

If these eigenvalues tell us the system will be unstable, we can re-generate the dynamics matrix, and perhaps reduce its variance.

We can also check the **trace** and **determinant** of our matrix to predict more about its behavior. Do so below. what do these values tell you?

In [ ]:
# fill in equations for the trace, determinant, and largest eigenvalue:

print('the trace is ' + '{:.5}'.format(___))
print('the determinant is ' + '{:.5}'.format(___))
print('the largest eigenvalue is ' + '{:.5}'.format(___))

Having reflected upon that, let's get to simulating.

You can drive your model with whatever signals you'd like - to start with, I'm giving it a three-dimensional noise input, two brief pulses of input to all units, and a five-second step input just to the integrator unit.

In [ ]:
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(121, projection='3d', aspect='auto')

integrator.clear_inputs()  # so that re-running this cell doesn't add inputs a second time

integrator.add_input_noise(0.1, weights=np.random.normal(0,1,(n_dims)))
integrator.add_input_noise(0.1, weights=np.random.normal(0,1,(n_dims)))
integrator.add_input_noise(0.1, weights=np.random.normal(0,1,(n_dims)))
integrator.add_pulse( 3, amplitude=1)
integrator.add_pulse(10, amplitude=1)

# add an input to the integrator
in_wts = np.zeros(n_dims)
in_wts[0] = 1
integrator.add_step( 20, time_off=25, amplitude=1.5, weights=in_wts)

integrator.plot_simulation(tmax=100, ax=ax, pca=True)

ax2 = fig.add_subplot(122)
integrator.plot_timeseries(ax=ax2, normalize=True)
plt.show()

As before, you can make neurons that are noisy observations of your dynamical system:

In [ ]:
ncells = 200  # however many you like
raster = integrator.observe_simulation(ncells, tmax=100, sigma=3)

fig = plt.figure(figsize=(7,5))
ax = fig.add_subplot(111)
plt.imshow(raster.T, aspect='auto')
plt.colorbar();

### 2.1.2 🏃 Picking the size of the fit dynamical system

Our integrator model has 20 dynamical units- but is it a 20-dimensional system? Use PCA on the activity you generated above to decide how many latent dimensions you need to "adequately" capture what's going on in the model.

🤔 Should you run PCA on the observed activity or the latents? Does it matter?

In [ ]:
# look at the dimensionality of your observed dynamical system- perhaps with PCA?

## 2.1.3 🏃 Fitting the input-driven rSLDS model
If you've picked a number of latent dimensions you're happy with, now is the time to fit your model. We just need to pull the model input from our `integrator` object. This gets passed as an additional argument `inputs` in two places: `ssm.initialize` and `ssm.fit`.

In [ ]:
# Global parameters you'll need to set:
D_obs = _                # number of observed dimensions
D_latent = _             # number of latent dimensions - up to you!
K = _                    # number of latent states to fit - up to you!
M = _                    # number of input dimensions - up to you!


# re-generate the input that was fed to our model, without noise.
timestamps = range(0, len(integrator.last_sim[0]))
in_sig = np.array([integrator._input(t*integrator.dt, noise=False) for t in timestamps])

# assuming you kept my example inputs, we can just keep the input to the first 2 units
model_input = in_sig[:,:2]


# Now fit that model:
# note that we have a new parameter M that should be used when initializing the SLDS model, using a keyword argument (M=M)

## 2.1.4 🏃 Plotting the ELBO and flow field

As for the Lorenz system, we should check our ELBOs to make sure the model has converged. If it has, then we can go ahead and examine the fit system.

In [ ]:
# you know the drill- plot that ELBO

How should we visualize the system? Here are a couple options, but can you think of others?

In [ ]:
# get the values of xhat and zhat
xhat = _
zhat = _

# everything below here you can run as-is
# get the first 2 PC's of the "neural" data
pca_data   = PCA(n_components=2)
pcs_data   = pca_data.fit_transform(raster)

# get the first 2 PC's of the latent dimensions of the fit model
pca_model  = PCA(n_components=2)
pcs_model  = pca_model.fit_transform(xhat)


fig = plt.figure(figsize=(8,4))

# plot the original system
ax = fig.add_subplot(121)
plt.plot(pcs_data[:,0], pcs_data[:,1], lw=1)
plt.title('first 2 PCs of data')

# plot the inferred latent trajectories and states
ax2 = fig.add_subplot(122)
plot_trajectory(zhat, pcs_model, ax=ax2)
plt.title('most-likely state of fit latents')


# Make some plots to compare the PCs of the observed data to those of the fit system:
fig = plt.figure(figsize=(8,8))
ax2 = fig.add_subplot(211)
plt.plot(pcs_data[:,0], label='data')
plt.plot(pcs_model[:,0], label='fit')
plt.legend()
ax2 = fig.add_subplot(212)
plt.plot(pcs_data[:,1], label='data')
plt.plot(pcs_model[:,1], label='fit')
plt.legend()

# Finally, show the fit flow field of the model:
fig2 = plt.figure(figsize=(8,8))
ax = fig2.add_subplot(111)
plot_flattened_dynamics(model, pca_model, xhat, ax=ax)
plt.title('fit flow field');

### 2.1.5 🏃 Getting the inputs wrong

In the fits above, we knew exactly what the input to our model was- as a result, our inferred flow field mostly ignores the input-driven portions of the system trajectories (🤔 how can you see that in the flow field?)

🐸 What would happen if we didn't know the model inputs, or if we only knew a subset of them? what if we guessed the wrong format of the inputs? Try going back to your model above, and re-fitting with missing, incomplete, or incorrect inputs. What kind of effect does this have on the inferred model dynamics?

In [ ]:
# insert fitting and visualization code here (or just go back up and re-run the cells above)

### 2.1.6 🏃 Examining parameters of the fit model

Once you're happy with your model fit, you can dig into its components to study them in more detail.
Recall that our fit model has the form

$$
x_t = A_k x_{t-1} + V_k u_{t} + b_k
$$

the fit dynamics matrix $\mathbf{A}$, input weights $\mathbf{V}$, and offset term $\mathbf{b}$ of our model can be unpacked from your model:

In [ ]:
print(model.dynamics.As)
print(model.dynamics.bs)
print(model.dynamics.Vs)

How do the eigenvalues of your fit dynamics matrix (or matrices) compare to those of the original system?

In [ ]:
e, v = np.linalg.eig(integrator.A)
e2, v2 = __  # compute the eigenvalues and eigenvectors of your fit dynamics matrix here

# fill in the blanks:
print('the trace is ' + '{:.5}'.format(_))
print('the determinant is ' + '{:.5}'.format(_))
print('the largest eigenvalue of the model is ' + '{:.5}'.format(_))
print('the largest eigenvalue of the fit is ' + '{:.5}'.format(_))

fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(111)
plt.plot(e.real, e.imag,'.')
plt.plot(e2.real, e2.imag,'r*')
plt.xlabel('real part')
plt.ylabel('imaginary part')
plt.axvline(x=1, color='r', linestyle='--');

## 2.2 🐸 further RNN adventures

If we've made it this far, then things went well. Maybe too well?

Obviously we've only scratched the surface of things you can do with input-driven dynamical systems. And you may have noticed above that the eigenvalues of the fit dynamics matrix aren't quite capturing the dynamics of the full original system. What might improve this? What might make the fit worse?

Some ideas for iterating on the above model:
* Make a hand-designed dynamics matrix $\mathbf{A}$ of your neural system, by picking a point you want to simulate from the Poincaré diagram.
* Add nonlinearities to the RNN to introduce oscillatory or chaotic dynamics. How does the rSLDS handle these?
* Write a new input generation function for `simulators.driven` to allow you to more easily feed time series data into your RNN model.
* Train an RNN on a task (eg using FORCE) and then compare its dynamics and eigenvalues to those of a fit rSLDS.
* And of course, try fitting on some real neural data.

<div align=center>
    🤖🤖🤖 Happy simulating! 🤖🤖🤖
    </div>